# 环境安装与基本概念

使用一个工具，最先开始的就是安装了。本文件夹下主要针对pytorch进行介绍，也会补充一些tensorflow的安装等。对于本repo，简单安装CPU版本的库即可（README.md中说明的环境里已经有了）。

不过下面仍然记录了GPU版的Pytorch安装过程，以备参考。

## ubuntu18.04安装pytorch1.2

### 版本搭配  

ubuntu18.04+cuda10.0+cudnn7.6.4+anaconda_python3.7+pytorch1.2

``` Shell
$ sudo lshw -C display | grep product
product: GP102 [GeForce GTX 1080 Ti]
```

### 驱动安装

先检查GPU的兼容性，[官网](https://developer.nvidia.com/cuda-gpus)上查看compute capability，满足大于3或者3.5的要求即可。

然后保证安装到Ubuntu上的包是最新的，因此

``` Shell
$ sudo apt update
$ sudo apt upgrade #will ask you Y/n if you want to upgrade
``` 

关于NVIDIA 的驱动：

Ubuntu预装了GPU的通用驱动。这些驱动不是优化的。因此需要找到适合自己的GPU的最新的NVIDIA驱动。有几种选择：

- Nvidia PPA：非常好的选择。通过使用PPA中包含的驱动达到开箱即用的效果。
- Ubuntu Default recommended driver：Ubuntu能指出自己的电脑需要的NVIDIA驱动
- Nouveau：NVIDIA驱动的开源实现版本。
- Official NVIDIA site：和PPA的一样，但是不会自动更新，并且有时在更新、卸载或者安装中会报错。

推荐的，也是比较好的方式是使用NVIDIA PPA来安装驱动。因为有最新的NVIDIA官方驱动，也在Ubuntu上测试过，且安装过程比较平滑。

``` Shell
$ sudo add-apt-repository ppa:graphics-drivers/ppa
$ sudo apt-get update
``` 

此语句将PPA库添加到Ubuntu的包系统内（Ubuntu是一种Debian Linux的发行版，因此使用的是Debian的dpkg包系统，该系统提供给Ubuntu应用来安装。高级包工具（APT）使得我们能很容易的在终端与dpkg交互）。

接下来需要决定安装哪个版本的driver。去[PPA库网站](https://launchpad.net/~graphics-drivers/+archive/ubuntu/ppa)上，
网页拉到最下面检查版本，发现很多版本都可以；也可以在[官网](https://www.nvidia.cn/Download/index.aspx?lang=cn)上检查，官网会给出一个版本；

在命令行输入：

``` Shell
$ ubuntu-drivers devices
``` 

可以看到给出了几个版本的驱动。

结合三者我决定选择nvidia-driver-410，因为410是对应cuda10.0的版本。

（**注意**：2019年10月24日看官网，支持的cuda版本已经为10.1，所以可以安装nvidia-driver-430，不过以内容下仍安装cuda10.0的安装写的）

``` Shell
$ sudo apt install nvidia-driver-410
``` 

安装完毕之后需要重启电脑，直接命令行reboot即可，以使用新的显卡驱动：

``` Shell
$ reboot
``` 

重启之后，在命令行键入：

``` Shell
$ nvidia-smi
``` 

查看是否已成功安装，出现一些信息基本上就是安装成功了。

如果安装的是430，问题也不大，即显示的信息里说CUDA版本是10.1的时候，
根据别人的[经验](https://zhuanlan.zhihu.com/p/73787970 )，装cuda-10.0也是可以的。 

为什么一定是10.0,？因为pytorch目前（2019年10月1日）支持的cuda是10.0。

为了保证驱动正常，所以有必要暂停其升级，使用以下命令：

``` Shell
$ sudo apt-mark hold nvidia-driver-410
nvidia-driver-410 set on hold.
``` 

To reverse this operation run:

``` Shell
$ sudo apt-mark unhold nvidia-driver-410
``` 

### 安装CUDA

首先check下gcc和g++的版本。

``` Shell
$ gcc --version
gcc (Ubuntu 7.3.0–16ubuntu3) 7.3.0
$ g++ --version
g++ (Ubuntu 7.3.0–16ubuntu3) 7.3.0
```

安装cuda10.0和9.0不太一样。直接从[官网](https://developer.nvidia.com/cuda-10.0-download-archive?target_os=Linux&target_arch=x86_64&target_distro=Ubuntu&target_version=1804&target_type=runfilelocal)下载
（用deb安装时我的报错了，不知道什么错误，有可能是因为cuda和driver的版本不完全匹配，所以用runfile安装了）

有提到要安装下kernel headers。（如果某个程序需要内核提供的一些功能，它就需要内核来编译程序，这个时候用的上kernel heads）。但几个也都没说。

``` code
$ sudo apt install linux-headers-$(uname -r)
```

执行命令即可安装，发现已经安装了。

按照官网提示安装即可，注意因为之前已经安装过驱动了，所以这里提示问是否安装驱动时（Install NVIDIA Accelerated Graphics Driver for Linux-x86_64 XXX.XX），选择no，其他的都可以选择yes或者默认值即可。

``` code
$ sudo ./cuda_10.0.130_410.48_linux.run
```

重启下看看自己没有黑屏。

然后配置环境变量。

``` code
$ vim ~/.bashrc #打开配置文件
$ export PATH=/usr/local/cuda-10.0/bin:$PATH
$ export LD_LIBRARY_PATH=/usr/local/cuda-10.0/lib64:$LD_LIBRARY_PATH
$ export CUDA_HOME=/usr/local/cuda-10.0
$ source ~/.bashrc
```
如果没有vim，安装即可，在命令行中输入：vim，没有安装的话会有提示，按照提示安装即可。

最后验证cuda是否安装成功：

``` code
$ cd /usr/local/cuda-10.0/samples/1_Utilities/deviceQuery
$ sudo make
$ ./deviceQuery
```

### 安装CUDNN

官网下载最新版的对应cuda10.0版本的CUDNN。

两种方法都有：

- 下载tar压缩包的：cudnn安装较容易只需要把文件解压后拷贝进cuda根目录即可。
- 下载三个deb文件的（the runtime library, the developer library, and the code samples library for Ubuntu 18.04）：

``` code
$ sudo dpkg -i libcudnn7_7.6.4.38-1+cuda10.0_amd64.deb
$ sudo dpkg -i libcudnn7-dev_7.6.4.38-1+cuda10.0_amd64.deb
$ sudo dpkg -i libcudnn7-doc_7.6.4.38-1+cuda10.0_amd64.deb 
```

安装完成验证是否安装成功：
Go to the MNIST example code: 

``` code
$ cd /usr/src/cudnn_samples_v7/mnistCUDNN/
```

Compile the MNIST example:

``` code
$ sudo make clean && sudo make
```

Run the MNIST example:

``` code
$ ./mnistCUDNN 
```

If your installation is successful, you should see Test passed!
现在已经不需要将gcc和g++降到6.X版本就可以执行。 

### 安装python

基础环境搭建好了之后，安装anaconda，现在（2019年10月1日）pytorch已经支持python3.7了。
下载linux下的安装包，执行：

``` code
$ sh Anaconda3-2019.07-Linux-x86_64.sh   #后边的文件名称是你的安装包的名称
```

然后一路按照默认的设置安装即可。
如果询问是否添加路径到环境变量或者是是否添加conda init，都选yes。

安装完成以后，重启终端。
先输入：

``` code
$ source ~/.bashrc
```

再输入：

``` code
$ python
```

即可看到安装成功的anaconda。

### 安装pytorch

官网目前（2019年11月1日）已经是1.3版本了，对应cuda是10.1，这里安装的是cuda10.0，对应的pytorch版本是1.2，因此需要找[以前的版本](https://pytorch.org/get-started/previous-versions/)安装。版本对应上即可。使用conda安装的时候，可能会出现没有权限的情况，给自己的用户赋予conda和anaconda相关文件夹权限即可，命令如下（以我的为例，owen是用户名;/home/owen/.conda是我的conda的根目录）：

``` code
sudo chown -R owen /home/owen/.conda
sudo chown -R owen /home/owen/anaconda3
```

使用conda安装完毕后，检查是否安装成功：

``` code
$ python
$ import torch
```

能导入即成功。

可以先安装vscode，再测试下能不能使用GPU加速。
安装vscode直接去官网即可。然后下载对应的python插件即可使用。

``` code
$ sudo dpkg -i code_1.38.1-1568209190_amd64.deb
```

测试代码：

In [1]:
import torch as t
x = t.rand(5,3)
y = t.rand(5,3)
if t.cuda.is_available():
    x = x.cuda()
    y = y.cuda()
    print(x+y)

测试成功之后，就可以自己写程序试试了。

另外，有一点需要注意：自己在后续使用Ubuntu系统过程中可能会进行系统升级（apt-get upgrade），这可能会导致 Failed to initialize NVML: Driver/library version mismatch

这时候先重启下电脑（reboot下）可能就好了，先不要重新捣鼓自己的显卡驱动。

## win10下安装pytorch

这里安装cuda10.1及对应驱动，然后安装pytorch最新稳定版。首先如前所述，查看电脑的GPU是否足够支持pytorch和tensorflow： https://developer.nvidia.com/cuda-gpus#compute 

比如我的笔记本GeForce 940M	5.0 是支持的。查看自己的电脑的显卡可以通过右键“计算机”文件夹，然后点击“设备管理器”，然后找到“显示适配器”即可看到。

然后查看自己可下载的NVIDIA Driver： https://www.nvidia.com/download/index.aspx?lang=en-us ，不过在下载安装之前，先看看cuda10.1对应的driver版本是什么：https://developer.nvidia.com/cuda-toolkit-archive

选择10.1版之后可以看到要下载的文件名里有cuda10.1对应的nvidia driver的版本，比如我选择的是10.1 update2：https://developer.nvidia.com/cuda-10.1-download-archive-update2?target_os=Windows&target_arch=x86_64&target_version=10&target_type=exelocal ，可以看到名称是cuda_10.1.243_426.00_win10.exe ，也就是说驱动版本是426

接下来就去这个网站：https://www.geforce.com/drivers 上找自己电脑对应的驱动版本，在manual driver search里面选择自己的显卡和操作系统信息，然后点击“START SEARCH”即可搜出匹配的driver版本。可以看到我的搜出来最老的也是440.97了，也和426不一致，所以还是选择直接从安装系统时就安装的nvidia软件里去更新了，参考网站：https://www.geforce.cn/whats-new/articles/nvidia-update

右键桌面，选择“NVIDIA 控制面板”，即可进入NVIDIA的界面。然而我的“帮助”栏下面并没有更新项目。所以最后放弃，直接安装最新的版本了。

如果安装顺利，那么就继续。

根据 CUDA10.1 安装文档：https://docs.nvidia.com/cuda/archive/10.1/cuda-installation-guide-microsoft-windows/index.html

安装需要vs的支持，可以看到cuda10.1支持vs2019 16.x 及之前版本，所以先安装一个vs2019.

然后接下来就可以安装cuda了。

然后是cudnn的安装。

最后安装pytorch。

## 云服务器安装

这里记录下Google Cloud Platform engine的使用。

一般个人电脑的GPU都不太好或者较旧，不太方便安装相应驱动来作计算，所以使用云端GPU是一个不错的选择，当然也可以远程连接到本地服务器上的GPU，这里主要记录远程使用Google GPU的方式，主要参考了：[创建 PyTorch Deep Learning VM 实例](https://cloud.google.com/ai-platform/deep-learning-vm/docs/pytorch_start_instance)，国内厂商云的可以参考:[从0开始使用腾讯云GPU服务器](https://www.jianshu.com/p/4ead0b0b42b5)，应该都大同小异。

为了使用GPU，首先要确保自己的项目有GPU资源，前提是先要有一个项目，云端项目管理可以参考：[创建和管理项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects?hl=zh-CN)，新建一个项目，我这里命名为'hydro-dl'

根据文档提示，首先访问[“AI Platform Deep Learning VM Image Cloud Marketplace”页面](https://console.cloud.google.com/marketplace/details/click-to-deploy-images/deeplearning?_ga=2.234722203.405444819.1595854801-1610178343.1595854801&_gac=1.120803066.1595857195.CjwKCAjw9vn4BRBaEiwAh0muDNhF-_GfXm-K20WhndkppgEWeTrhU9PVpIdD3owiU6voEHEJ_ru71xoC_DsQAvD_BwE)，费用预估是一月300美元，可以先“启动”，回头再调整使用计费方式。点击之后可以看到刚刚创建的项目，选择hydro-dl，然后会提示需要创建结算账号，这样google才能收钱，就按照要求创建即可，这里结算账号我也命名为hydro-dl了，然后按顺序操作提交并启用结算功能即可，这时候再重新点击“启动”，然后选择hydro-dl项目，Google cloud就会配置Deep Learning VM了。

进入新的页面后，会提示GPU配额不足，需要现在配额页面增加配额，就根据提示前往配额页面，根据[资源配额](https://cloud.google.com/compute/quotas?_gac=1.46270037.1595857195.CjwKCAjw9vn4BRBaEiwAh0muDNhF-_GfXm-K20WhndkppgEWeTrhU9PVpIdD3owiU6voEHEJ_ru71xoC_DsQAvD_BwE&_ga=2.176034850.-1610178343.1595854801) 页面介绍，用户需要首先申请调整配额，进入配额页面之后，过滤条件搜索NVIDIA，因为创建的时候提示的是NVIDIA Tesla K80，所以选择K80，然后就可以看到一系列服务，我就选择us-east-1，然后点击修改配额，输入个人信息以供联系，然后提交申请，需要等待google回复，回复的第一封邮件说需要2天，不过我这个申请比较小，所以几分钟就有回复了。然后配额完成还需要15分钟，所以稍等一会儿再重新进入Deep Learning VM 去部署，可以重新查看配额，还是搜索NVIDIA，选择K80，应该可以看到现在的 us-east-1 的那一条对应的限制已经是1了，说明修改完成，可以继续部署了。

很奇怪依然显示我的配额不足，那就暂时先不用这种“AI Platform Deep Learning VM Image Cloud Marketplace”的方式了，直接在compute engine上添加GPU来使用下看看到底是哪里有问题。

首先，首先，安装 gcloud命令行工具到最新版本，参考:[gcloud compute](https://cloud.google.com/compute/docs/gcloud-compute?hl=zh-cn)。首先安装cloud SDK，按提示下载安装即可，需要等待一小段时间，最后完成时候，出现的选项应是全部默认勾选的，按默认的即可，这样可以进入命令行配置界面，提示需要登陆，选择Y，然后进入登陆网页，点击自己的google 账号，然后允许SDK权限申请即可使用Cloud SDK了。（不适用SDK也可以配置GPU）

接下来，完成一个虚拟机创建：[添加或移除 GPU](https://cloud.google.com/compute/docs/gpus/add-gpus?hl=zh-cn)，[Linux 虚拟机使用快速入门](https://cloud.google.com/compute/docs/quickstart-linux)，前面已经完成了项目创建和结算启用，因为我已经启动了compute engine，所以这一步也不用再做了。

转到虚拟机实例页面，点击创建实例，设置实例名称为hydro-dl，要使用GPU，所以先确认下GPU在各地区的情况：[Compute Engine 上的 GPU](https://cloud.google.com/compute/docs/gpus?_ga=2.226450335.405444819.1595854801-1610178343.1595854801&_gac=1.50456027.1595857195.CjwKCAjw9vn4BRBaEiwAh0muDNhF-_GfXm-K20WhndkppgEWeTrhU9PVpIdD3owiU6voEHEJ_ru71xoC_DsQAvD_BwE)，各个GPU都有，所以默认的us-central1可以。默认的us-central1-a尝试了之后发现资源不足，创建不成功，所以选择us-central1-c。CPU我选择两个标准CPU 7.5G内存的，点击“CPU 平台和 GPU”标签，会显示添加GPU的选项。点击“添加GPU”，默认的K80即可。因为我更熟悉Ubuntu，所以将启动磁盘设置为Ubuntu，选择系统Ubuntu，然后版本选择18.04，磁盘也不贵，先选择300G用用。点击允许http流量，然后点击创建，发现提示配额不足，GPUS_ALL_REGIONS 有限制，说明需要的是GPUS_ALL_REGIONS的配额，所以还是需要重新去配，在配额界面搜索 GPUS_ALL_REGIONS，然后申请将限制改为1即可，再重新进入配额页面搜索，可以发现 GPUS_ALL_REGIONS 限制已经变为1了。现在按照前面的步骤重新创建实例，完成后即可得到一台远程的有GPU的机器了。不过好像直接创建还是不行，所以先等待15分钟，再试试。创建完成之后，就可以尝试远程连接了。

先不连接，因为如果自己去设置GPU驱动等还是有点麻烦的，所以现在重新回到[“AI Platform Deep Learning VM Image Cloud Marketplace”页面](https://console.cloud.google.com/marketplace/details/click-to-deploy-images/deeplearning?_ga=2.234722203.405444819.1595854801-1610178343.1595854801&_gac=1.120803066.1595857195.CjwKCAjw9vn4BRBaEiwAh0muDNhF-_GfXm-K20WhndkppgEWeTrhU9PVpIdD3owiU6voEHEJ_ru71xoC_DsQAvD_BwE)，尝试直接使用deep learning VM。

现在进入页面，发现就没有配额限制的问题了。命名hydro-dl，地区还选择us-central1-c，然后framwork选择pytorch，勾选驱动安装以及jupyter，硬盘还选择300g，点击部署，等待即可，这会比之前部署的时候时间要久一些。生成之后进入到实例页面即可看到刚刚部署好的实例。

远程连接实例可以参考：[连接到实例](https://cloud.google.com/compute/docs/instances/connecting-to-instance?hl=zh-cn)。在实例页面，点击SSH即可通过浏览器远程连接了。如果需要使用第三方工具，可以参考：[使用第三方工具进行连接](https://cloud.google.com/compute/docs/instances/connecting-advanced#thirdpartytools)。这里使用mobaxterm尝试连接，参考了：[How to connect to google cloud over Mobaxterm SSH](https://www.youtube.com/watch?v=J4fJ3m_nAJ0)以及[Create Linux VM Instance on Google Cloud Platform & Access Via SSH From Windows](https://www.youtube.com/watch?v=Rs7HMMVDqC8)

首先，在实例页面复制自己的google cloud的外部IP；

然后第三方工具都必须先生成自己的 SSH 密钥对，并将您的 SSH 公钥文件提供给实例，然后才能进行连接，所以这里第二步用 mobakeygen 工具来生成SSH key，选择tools峡的mobakeygen，然后点击generate，再随意移动自己的鼠标即可生成，生成后点击“save private key”,yes之后存入自己想放的位置，我命名了private_key；

然后copy自己的public key，在实例页面，选择左边栏的“元数据”，点击"SSH密钥"，点击“修改”，然后添加刚刚复制的公钥，并将最后的一串字符，我的是“rsa-key-20200727”改成自己的谷歌账号邮箱：xxx@gmail.com，然后保存即可；

接下来，再在云端使用ssh-keygen生成keygen，点击实例页面的"SSH"进入网页远程界面，在打开的界面输入 ssh-keygen，然后给文件命名，我命名为：google_cloud_ssh，密码直接两个回车跳过即可。然后命令行输入ls -la即可看到刚刚生成的ssh文件。copy google_cloud_ssh 到 .ssh/下：cp google_cloud_ssh .ssh/ ，然后进入.ssh文件夹输入ls可以看到copy进来的文件：先 cd .ssh ，然后 ls

最后就可以创建ssh连接了，点击session图标，然后选择SSH，在Remote host一栏写入：IP Address，即自己实例的外部IP，然后在Advanced SSH settings 里面导入刚刚保存的私钥，确定即可连接，然后输入用户名登陆，刚刚自己添加邮箱的时候的用户名：xxx@gmail.com里的xxx。即可进入页面。

如果没有用google提供好的镜像，久要自己安装nvidia驱动以及cuda，cudnn了，云端安装和本地略有不同，需要参考：[安装 GPU 驱动程序](https://cloud.google.com/compute/docs/gpus/install-drivers-gpu?hl=zh-cn#ubuntu-driver-steps)。这里暂时没有尝试了。

连接到之后可以输入python, conda env list, nvidia等可以看到已经配置好的内容。接下来就可以尝试自己的项目了。